In [1]:
import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from numpy import load
from torch.utils.data import Dataset

In [2]:
# load array
Data = load('Data_09.npy')
# print the array

In [3]:
Data=np.nan_to_num(Data)

In [4]:
Xdata=[]
Ydata=[]
Xtrain_data=[]
Ytrain_data=[]
Xtest_data=[]
Ytest_data=[]

for idx in range(len(Data)//39):
#     print(np.isinf(Data[39*idx:39*(idx)+38][:,:].tolist()).any())
    if(np.isinf(Data[39*idx:39*(idx)+38][:,:].tolist()).any()):
        print(np.isinf(Data[39*idx:39*(idx)+38][:,:].tolist()).any())
        continue
    if idx< 3735:
        Xtrain_data.append(Data[39*idx:39*(idx)+39][:,1:].tolist())
        Ytrain_data.append(Data[39*idx:39*(idx)+39][:,-1].tolist())
    else:
        Xtest_data.append(Data[39*idx:39*(idx)+39][:,1:].tolist())
        Ytest_data.append(Data[39*idx:39*(idx)+39][:,-1].tolist())

Xtrain_data=np.vstack(Xtrain_data)
Ytrain_data=np.vstack(Ytrain_data)

Xtrain_data=torch.FloatTensor(Xtrain_data)
Ytrain_data=torch.IntTensor(Ytrain_data)
Ytrain_data=Ytrain_data.view(-1)


Xtest_data=np.vstack(Xtest_data)
Ytest_data=np.vstack(Ytest_data)
Xtest_data=torch.FloatTensor(Xtest_data)
Ytest_data=torch.IntTensor(Ytest_data)
Ytest_data=Ytest_data.view(-1)

In [5]:
Xtrain_data.size()

torch.Size([145665, 6])

In [6]:
Ytrain_data=2*(Ytrain_data>0).long()+(Ytrain_data==0).long()
Ytest_data=2*(Ytest_data>0).long()+(Ytest_data==0).long()

Ydata=(torch.stack([Ydata>0,Ydata==0, Ydata<0],axis=0)).float()

In [7]:
Ytrain_data=Ytrain_data.T
Ytest_data=Ytest_data.T

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def batchify(data, bsz):
    # 데이터셋을 bsz 파트들로 나눕니다.
    nbatch = data.size(0) // bsz
    # 깔끔하게 나누어 떨어지지 않는 추가적인 부분(나머지들) 은 잘라냅니다.
    data = data.narrow(0, 0, nbatch * bsz)
    # 데이터에 대하여 bsz 배치들로 동등하게 나눕니다.
    data = data.view(bsz, -1,data.size(1)).transpose(0,1).contiguous()
    return data.to(device)

# batch_size = 25
# eval_batch_size = 25

# Xtrain_data = batchify(Xtrain_data, batch_size)
# Xtest_data = batchify(Xtest_data, eval_batch_size)
# Ytrain_data = batchify(Ytrain_data, batch_size)
# Ytest_data = batchify(Ytest_data, eval_batch_size)

In [9]:
bptt = 39
def get_batch(source, i,bs):
    seq_len = min(bptt*bs, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].reshape(-1)
    return data, target

In [10]:
from torch import Tensor
import torch
import torch.nn as nn
from torch.nn import Transformer
import math
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

INPUT_SIZE=6
EMB_SIZE=64
HIDDEN_SIZE=64
NUM_LAYERS=6
PROJ_SIZE=3
BATCH_SIZE=16

In [11]:
class Seq2SeqLSTM(nn.Module):
    def __init__(self,
                 input_size: int,
                 hidden_size: int,
                 num_layers: int,
                 emb_size:int,
                 proj_size:int):
        
        super(Seq2SeqLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size=emb_size,hidden_size=hidden_size,num_layers=num_layers)
        self.generator = nn.Linear(emb_size, proj_size)
        self.src_tok_emb = nn.Linear(input_size, emb_size)
        self.bn1=nn.BatchNorm1d(BATCH_SIZE)
    def forward(self,
                src: Tensor):

        src_emb = self.src_tok_emb(src)
        src_emb=self.bn1(src_emb)
#         outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
#                                 src_padding_mask, tgt_padding_mask, memory_key_padding_mask)


        outs,_ = self.lstm(src_emb)
#         print(outs.size())
        return self.generator(outs)

In [12]:
torch.manual_seed(0)
lstm=Seq2SeqLSTM(input_size=INPUT_SIZE,hidden_size=HIDDEN_SIZE,num_layers=NUM_LAYERS,emb_size=EMB_SIZE,proj_size=PROJ_SIZE)
for p in lstm.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)
lstm = lstm.to(DEVICE)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(lstm.parameters(), lr=1e-5, betas=(0.9, 0.98), eps=1e-9)

In [13]:
Xtrain_data[0]

tensor([1.7992e-02, 1.9043e-02, 0.0000e+00, 5.0380e+04, 4.6628e+04, 0.0000e+00])

In [14]:
from torch.utils.data import DataLoader

def train_epoch(model, optimizer):
    model.train()
    losses = 0
    
    total=0
    correct=0
    correct1=0
    tot1=0
    correct2=0
    tot2=0
    correct0=0
    tot0=0
    for batch, i in enumerate(range(0, Xtrain_data.size(0) - 1, BATCH_SIZE*bptt)):
        data, org_targets = get_batch(Xtrain_data, i,BATCH_SIZE)
        if (data.isnan().any() or data.isinf().any()):
#            print(data)
            continue
#         _,targets = get_batch(Ytrain_data,i)
        targets,_ = get_batch(Ytrain_data,i,BATCH_SIZE)
#         src = src.to(DEVICE)
#         tgt = tgt.to(DEVICE)
        targets=torch.unsqueeze(targets,1)
        src=batchify(data,BATCH_SIZE)
        tgt=batchify(targets,BATCH_SIZE)
        src=src[:-1]
        tgt=tgt[1:]
        logits = model(src)

        optimizer.zero_grad()

        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt.reshape(-1))
        loss.backward()

        optimizer.step()
        losses += loss.item()
        
        _,predicted=torch.max(logits,-1)
        
        correct += (tgt.squeeze() == predicted).sum().item()
        total+=len(predicted)*BATCH_SIZE
        tot0+=(0== tgt.squeeze()).sum().item()
        tot1+=(1== tgt.squeeze()).sum().item()
        tot2+=(2== tgt.squeeze()).sum().item()
        correct0+=((0== predicted) &(0==tgt.squeeze())).sum().item()
        correct1+=((1== predicted)&(1==tgt.squeeze())).sum().item()
        correct2+=((2== predicted)&(2==tgt.squeeze())).sum().item()
            

    print("TrainAcc:",correct/total*100)            
    print("TrainAcc0:",correct0/tot0*100,correct0,tot0)     
    print("TrainAcc1:",correct1/tot1*100,correct1,tot1)     
    print("TrainAcc2:",correct2/tot2*100,correct2,tot2)     
    return losses / Xtrain_data.size(0)


def evaluate(model):
    model.eval()
    losses = 0
    total=0
    correct=0
    correct1=0
    tot1=0
    correct2=0
    tot2=0
    correct0=0
    tot0=0
    with torch.no_grad():
        for i in range(0, Xtest_data.size(0) - 1, bptt):

            data, org_targets = get_batch(Xtest_data, i,BATCH_SIZE)
            if (data.isnan().any() or data.isinf().any()):
#            print(data)
                continue
            targets,_ = get_batch(Ytest_data,i,BATCH_SIZE)
            targets=torch.unsqueeze(targets,1)
            src=batchify(data,BATCH_SIZE)
            tgt=batchify(targets,BATCH_SIZE)

            logits = model(src)

            loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt.reshape(-1))
            if(loss.isnan()):
                print(src,tgt_input)
                break
            losses += loss.item()
            _,predicted=torch.max(logits,-1)

            
            correct += (tgt.squeeze() == predicted).sum().item()
            total+=len(predicted)*BATCH_SIZE
            tot0+=(0== tgt.squeeze()).sum().item()
            tot1+=(1== tgt.squeeze()).sum().item()
            tot2+=(2== tgt.squeeze()).sum().item()
            correct0+=((0== predicted) &(0==tgt.squeeze())).sum().item()
            correct1+=((1== predicted)&(1==tgt.squeeze())).sum().item()
            correct2+=((2== predicted)&(2==tgt.squeeze())).sum().item()
            
    print(total)
    print(correct)
    print("Acc:",correct/total*100)            
    print("Acc0:",correct0/tot0*100,correct0,tot0)     
    print("Acc1:",correct1/tot1*100,correct1,tot1)     
    print("Acc2:",correct2/tot2*100,correct2,tot2)     
    return losses / Xtest_data.size(0)

In [15]:
evaluate(lstm)

460064
87316
Acc: 18.97909856020032
Acc0: 100.0 87316 87316
Acc1: 0.0 0 301695
Acc2: 0.0 0 71053


0.02867435404200333

In [16]:
from timeit import default_timer as timer
NUM_EPOCHS = 1000
best_val_loss=100000000
for epoch in range(1, NUM_EPOCHS+1):
    start_time = timer()
    train_loss = train_epoch(lstm, optimizer)
    end_time = timer()
    val_loss = evaluate(lstm)
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "f"Epoch time = {(end_time - start_time):.3f}s"))
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = lstm
PATH='best_model_seq'
torch.save(best_model.state_dict(), PATH)

TrainAcc: 21.48393461104848
TrainAcc0: 100.0 30490 30490
TrainAcc1: 0.0 0 82568
TrainAcc2: 0.0 0 28862
460064
87316
Acc: 18.97909856020032
Acc0: 100.0 87316 87316
Acc1: 0.0 0 301695
Acc2: 0.0 0 71053
Epoch: 1, Train loss: 0.002, Val loss: 0.027, Epoch time = 4.785s
TrainAcc: 52.06242953776775
TrainAcc0: 17.93046900623155 5467 30490
TrainAcc1: 82.86503245809514 68420 82568
TrainAcc2: 0.0 0 28862
460064
285269
Acc: 62.00637302636155
Acc0: 8.069540519492419 7046 87316
Acc1: 92.21995724158505 278223 301695
Acc2: 0.0 0 71053
Epoch: 2, Train loss: 0.002, Val loss: 0.026, Epoch time = 4.792s
TrainAcc: 56.37612739571589
TrainAcc0: 6.097081010167268 1859 30490
TrainAcc1: 94.64925879275265 78150 82568
TrainAcc2: 0.0 0 28862
460064
290535
Acc: 63.15099638311191
Acc0: 5.3655687388336615 4685 87316
Acc1: 94.74800709325643 285850 301695
Acc2: 0.0 0 71053
Epoch: 3, Train loss: 0.002, Val loss: 0.025, Epoch time = 4.766s
TrainAcc: 57.05326944757609
TrainAcc0: 3.7782879632666444 1152 30490
TrainAcc1: 9

KeyboardInterrupt: 